In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_json('prepped_data.json').sort_index()
df.head()

,age,num_comments,score,text,timestamp,weekday_posted,hour_posted,log_score,log_comments
0,300,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08,2,11,9.985989,7.370237
1,240,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31,2,12,7.886457,5.996477
2,660,3320,46977,Women outperform men after Japan medical schoo...,2019-06-19 05:51:44,2,5,10.757413,8.107723
3,360,202,1474,MH17 crash: Investigators 'to charge four with...,2019-06-19 10:50:51,2,10,7.295735,5.308317
4,660,1336,2665,Iranian official calls on world to unite again...,2019-06-19 05:09:15,2,5,7.887959,7.197443


In [3]:
pipe = Pipeline(steps=[
    ('vec', TfidfVectorizer()),
    ('model', LinearRegression())
])

In [5]:
pipe.fit(df['text'], df['log_score'])

Pipeline(memory=None,
     steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...('model', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])

In [9]:
pipe.predict(df['text']).shape == df['log_score'].shape

True

tfidf will work in a pipe (despite today's earlier boofs)  
Now to try text and remaining features

In [11]:
text_transformer = Pipeline(steps=[
    ('vec', TfidfVectorizer())
])

In [12]:
preprocessor = ColumnTransformer(transformers=[
    ('text', text_transformer, df[['text']].columns)
])

In [13]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [45]:
test_array = np.array([['hello', 'goodbye'],['killME!', 'TFIDF!!']])
print(test_array)
print('len', len(test_array))
print('shape', test_array.shape)
print()
test_frame = pd.DataFrame(test_array, columns=['text', 'nope'])
display(test_frame)
print('len', len(test_frame))
print('shape', test_frame.shape)
print()
test_target = np.array([[1],[10]])
display(test_target)
print('len', len(test_target))
print('shape', test_target.shape)
print('type of tsttarget', type(test_target))
print()
pipe.fit(test_frame, test_target)
test_frame

[['hello' 'goodbye']
 ['killME!' 'TFIDF!!']]
len 2
shape (2, 2)



,text,nope
0,hello,goodbye
1,killME!,TFIDF!!


len 2
shape (2, 2)



array([[ 1],
       [10]])

len 2
shape (2, 1)
type of tsttarget <class 'numpy.ndarray'>



ValueError: Found input variables with inconsistent numbers of samples: [1, 2]

In [46]:
test_array

array([['hello', 'goodbye'],
       ['killME!', 'TFIDF!!']], dtype='<U7')

In [33]:
pipe.predict(test_frame,)

TypeError: predict() takes 2 positional arguments but 3 were given

In [80]:
# pipe.fit(X=df['text'], y=df['score'])    # IndexError: tuple index out of range

In [87]:
# pipe.fit(X=df[['text']], y=df['score'])     # ValueError: Found input variables with inconsistent numbers of samples: [1, 4842]

In [82]:
# pipe.fit(X=pd.concat([df['text'],df['text']], axis=1), y=df['score']) # ValueError: Found input variables with inconsistent numbers of samples: [2, 4842]

In [95]:
preprocessor = ColumnTransformer(transformers=[
    ('text', TfidfVectorizer(), df[['text']].columns)
])
preprocessor.fit_transform(pd.concat([df['text'], df['text']],axis=1))

array([[1.],
       [1.]])

In [91]:
df['text'].shape

(4842,)

Test for vectorizer and transformer

In [65]:
tfidf_tran = TfidfTransformer()
tfidf_vec = TfidfVectorizer()

In [70]:
tfidf_vec.fit(df.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

THIS WAS REALLY GOOD WORK ... TFIDF just won't cooperate with ColumnTransformer for anyone here.